# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Описание данных

 - id — уникальный идентификатор скважины;
 - f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
 - product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from numpy.random import RandomState

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

Загрузим данные:

In [2]:
data_0 = pd.read_csv('geo_data_0.csv')
data_1 = pd.read_csv('geo_data_1.csv')
data_2 = pd.read_csv('geo_data_2.csv')

Посмотрим на них:

In [3]:
data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
data_1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [5]:
data_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


Во всех датасетах присутствует id скважины - для обучения и предсказания этот столбец нам не нужен

In [6]:
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

И посмотрим на информацию об этих данных:

In [7]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


In [8]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


In [9]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


Пропусков нет, числа в формате float, все ок. Перейдем к следующему пункту

## Обучение и проверка модели

### Для региона 0:

Выделим признаки и цель:

In [10]:
features_0 = data_0.drop(['product'], axis=1)
target_0 = data_0['product']

Разобьем на выборки:

In [11]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = \
        train_test_split(features_0, target_0, test_size=0.25, random_state=12345)

Создадим и обучим модель:

In [12]:
model_0 = LinearRegression()

In [13]:
model_0.fit(features_train_0, target_train_0)

LinearRegression()

In [14]:
predictions_valid_0 = model_0.predict(features_valid_0)

Посчитаем MSE и RMSE:

In [15]:
mse_0 = mean_squared_error(target_valid_0, predictions_valid_0)
rmse_0 = mean_squared_error(target_valid_0, predictions_valid_0)**0.5

In [16]:
print('MSE =', mse_0)
print('RMSE =', rmse_0)

MSE = 1412.2129364399243
RMSE = 37.5794217150813


Проверим модель на адекватность с помощью коэффициента детерминации:

In [17]:
r2_score_0 = r2_score(target_valid_0, predictions_valid_0)
print("R2 =", r2_score_0)

R2 = 0.27994321524487786


Это не 0, но и не единица. То есть не среднее, но и не идеал.

Можно посчитать и среднее:

In [18]:
m_0 = target_train_0.mean() # среднее
predicted_valid_0 = pd.Series(target_train_0.mean(), index=target_valid_0.index)
mse_0 = mean_squared_error(target_valid_0, predicted_valid_0)

In [19]:
print("Mean =", m_0)
print("MSE =", mse_0)
print("RMSE =", mse_0 ** 0.5)

Mean = 92.6404677530568
MSE = 1961.5678757223516
RMSE = 44.289591053907365


Посчитаем среднее предсказание для каждого региона:

In [20]:
mean_product_0 = predictions_valid_0.mean()
print(mean_product_0)

92.59256778438038


Сделаем из предсказаний объект Series

In [21]:
predictions_valid_0 = pd.Series(predictions_valid_0, index=target_valid_0.index) 

**Cделаем то же самое для других регионов:**

### Для региона 1:

Признаки и цель, разбиемние на выборки:

In [22]:
features_1 = data_1.drop(['product'], axis=1)
target_1 = data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = \
        train_test_split(features_1, target_1, test_size=0.25, random_state=12345)

Модель:

In [23]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)

Качество:

In [24]:
mse_1 = mean_squared_error(target_valid_1, predictions_valid_1)
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5

In [25]:
print('MSE =', mse_1)
print('RMSE =', rmse_1)

MSE = 0.7976263360391137
RMSE = 0.8930992867756158


In [26]:
r2_score_1 = r2_score(target_valid_1, predictions_valid_1)
print("R2 =", r2_score_1)

R2 = 0.9996233978805127


Среднее:

In [27]:
mean_product_1 = predictions_valid_1.mean()
print(mean_product_1)

68.728546895446


In [28]:
predictions_valid_1 = pd.Series(predictions_valid_1, index=target_valid_1.index) 

### Для региона 2:

In [29]:
features_2 = data_2.drop(['product'], axis=1)
target_2 = data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = \
        train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

In [30]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)

Качество:

In [31]:
mse_2 = mean_squared_error(target_valid_2, predictions_valid_2)
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2)**0.5

In [32]:
print('MSE =', mse_2)
print('RMSE =', rmse_2)

MSE = 1602.3775813236196
RMSE = 40.02970873393434


In [33]:
r2_score_2 = r2_score(target_valid_2, predictions_valid_2)
print("R2 =", r2_score_2)

R2 = 0.20524758386040443


Среднее:

In [34]:
mean_product_2 = predictions_valid_2.mean()
print(mean_product_2)

94.96504596800489


In [35]:
predictions_valid_2 = pd.Series(predictions_valid_2, index=target_valid_2.index) 

### Итоги

Давайте теперь создадим таблицу с результатами расчетов:

In [36]:
mse_final = [mse_0, mse_1, mse_2]
rmse_final = [rmse_0, rmse_1, rmse_2]
r2_score_final = [r2_score_0, r2_score_1, r2_score_2]
mean_product = [mean_product_0, mean_product_1, mean_product_2]

Создадим датафрейм:

In [37]:
result = {'mse' : mse_final, 'rmse' : rmse_final, 'r2_score' : r2_score_final, 'mean': mean_product}
result = pd.DataFrame(data=result) 

In [38]:
result

,mse,rmse,r2_score,mean
0,1961.567876,37.579422,0.279943,92.592568
1,0.797626,0.893099,0.999623,68.728547
2,1602.377581,40.029709,0.205248,94.965046


### Выводы

- Итак, мы обучили три модели: по каждому региону своя. Для региона "1" модель получилась идеальной - коэффициент детерминации почти равен единице, для других двух коэффициент детерминации больше 0, что говорит о не самом плохом качестве модели. 

- средний запас предсказанного сырья у региона "2" наибольший и равен 94.96 тыс. баррелей. Но и качество модели там наихудшее.

- У региона "1" с  наилучшей моделью запасы самые маленькие - 68.73 тыс. баррелей

## Подготовка к расчёту прибыли

Сохраним ключевые значения для расчетов в отдельных переменных:

In [39]:
BUDGET = 10**10 # бюджет в 10 млрд
BARREL = 450 # 450р за баррель
N = 200 # количество учитываемых скважин
BARREL_THOUSAND = BARREL * 1000

Рассчитаем объем сырья для безубыточной разработки новой скважины:

In [40]:
VOLUME = BUDGET / BARREL/ N # необходимый объем для "выхода в ноль"

Теперь у нас есть количество требуемой нефти в баррелях, а у нас в таблице указаны тыс. баррелей. Поэтому разделим на 1000

In [41]:
print('Объем сырья для безубыточной разработки: {}'.format(round(VOLUME / 1000, 2)))

Объем сырья для безубыточной разработки: 111.11


То есть для безубыточной добычи на 200 скважинах нам необходимо, чтобы сумма запасов была 111,11 тыс. баррелей в каждой

Давайте посмотрим на средние значения по регионам:

In [42]:
print('Регион 0:', round(data_0['product'].mean(),2))
print('Регион 1:', round(data_1['product'].mean(),2))
print('Регион 2:', round(data_2['product'].mean(),2))

Регион 0: 92.5
Регион 1: 68.83
Регион 2: 95.0


Таким образом, средние значения запасов нефти в регионах меньше необходимых для получения прибыли

## Расчёт прибыли и рисков 

Из прошлых пунктов имеем:

- predictions_valid_0, predictions_valid_1, predictions_valid_2 - предсказания
- mean_product_0, mean_product_1, mean_product_2 - средние значения предсказаний
- barrel_thousand
- volume
- budget

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [43]:
def profit(predictions, target): # на вход подаются предсказания и целевые признаки
    predictions = pd.Series(predictions,index=target.index) # для начала нам нужно, чтобы инексы совпадали
    max_predictions = predictions.sort_values(ascending=False)[:200] # 200 наибольших
    total = sum(target[max_predictions.index]) # суммируем 200 целевых признаков, которые соответствуют предсказаниям
    profit = total * BARREL_THOUSAND - BUDGET # считаем прибыль
    return profit

### Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

Если я все правильно понял, мы должны взять 1000 выборок из каждого региона, в каждой выборке 500 скважин

In [44]:
state = np.random.RandomState(12345)
def function(predictions,target):
    profit_list = [] # список значений прибыли для каждой выборки 
    for i in range(1000): # количество выборок
        subsample = target.sample(n=500, replace=False, random_state=state) # создаем подвыбооку  по целевым признакам из 500 скважин
        profit_list.append(profit(predictions, subsample)) # применяем нашу функцию и добавляем ее результат в список
    profit_list = pd.Series(profit_list)
    lower = profit_list.quantile(q=0.025)
    upper = profit_list.quantile(q=0.975)
    print('Доверительный интвервал: от {l} до {u} млн рублей'.format(l=round(lower / 1000000), u=round(upper / 1000000)))
    print('Риск: {}%'.format(round(profit_list.loc[profit_list < 0].count() / len(profit_list) * 100, 2)))
    print('Риск другой формулой: {}%'.format((profit_list < 0).mean() * 100))

In [45]:
function(predictions_valid_0, target_valid_0) # для региона 0

Доверительный интвервал: от -127 до 880 млн рублей
Риск: 7.2%
Риск другой формулой: 7.199999999999999%


In [46]:
function(predictions_valid_1, target_valid_1) # для региона 1

Доверительный интвервал: от 47 до 840 млн рублей
Риск: 1.3%
Риск другой формулой: 1.3%


In [47]:
function(predictions_valid_2, target_valid_2) # для региона 2

Доверительный интвервал: от -116 до 907 млн рублей
Риск: 7.3%
Риск другой формулой: 7.3%


## Выводы

Таким образом, были посчитаны риски и прибыль для трех регионов:

    - самые маленькие риски у региона 1 - 1,3%. Так же у данного региона границы 95% доверительного интервала расположены ближе друг к другу, нежели в других регионах (от 47 до 840 млн. рублей)
    - самые большой риск - у региона 2 - 7,3%, но и максимально возможная прибыль тоже довольна высока - 907 млн. рублей
    - у региона 0 риск составляет 7,2%, а доверительный интервал - от -127 до 880 млн рублей
    
Для разработки скважин я бы выбрал регион 1, из-за минимальных рисков и более узкого доверительного интервала    